In [ ]:
import os
os.environ['CUDA_DIVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='1'

import librosa
import librosa.display
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean
from specgrams_helper import SpecgramsHelper

fname = 'sample.wav'

In [ ]:
def one_to_three(one_dim):
    two_dim = np.expand_dims(one_dim, axis=0)
    three_dim = np.expand_dims(two_dim, axis=-1)
    return three_dim

In [ ]:
audio_original, sample_rate = librosa.load(
    path='./' + fname,
    sr=16000,
    mono=True,
    duration=None
)

## Define SpecgramsHelper class instance

In [ ]:
A = SpecgramsHelper(
    audio_length=64000,
    spec_shape=(256, 512),
    overlap=0.75,
    sample_rate=16000,    
    mel_downscale=2
)

## Convert Wave file to STFT spectrogram

In [ ]:
tensor_specgram = A.waves_to_specgrams(one_to_three(audio_original))
specgram = tf.Session().run(tensor_specgram)

In [ ]:
logmag = specgram[0, :, :, 0]
phase = specgram[0, :, :, 1]

print("Log Mag shape: \t\t" + str(logmag.shape))
print("Log Phase shape: \t" + str(phase.shape))

librosa.display.specshow(
    logmag,
    sr=sample_rate,
    fmax=sample_rate//2
)
plt.title("Log Magnitude")
plt.tight_layout()
plt.show()

librosa.display.specshow(
    phase,
    sr=sample_rate,
    fmax=sample_rate//2
)
plt.title("Log Phase")
plt.tight_layout()
plt.show()

## Convert Spectrogram to STFT matrix

In [ ]:
tensor_stft = A.specgrams_to_stfts(tensor_specgram)
tensor_wav = A.stfts_to_waves(tensor_stft)
audio_restored = tf.Session().run(tensor_wav)[0,:,0]

# librosa.output.write_wav('./restored.wav', audio_restored, sample_rate)
dist = euclidean(audio_original, audio_restored)
print(dist)

In [ ]:
from pre_process import PreProcessor

dist_list = []
wav_dir = './NSynth'

P = PreProcessor()
P.get_dist_meanstd(wav_dir, dist_list=dist_list)